# Penguins in Production - Session 2

This program creates a [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) to build an end-to-end Machine Learning system to solve the problem of classifying penguin species. This example uses the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data).

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset' width="900">

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) we built in the previous session with a step to train a model. We'll explore the [Training](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) and the [Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) steps. 

Here is what the Pipeline will look like at the end of this session:

<img src='images/session2-pipeline.png' alt='Session 2 Pipeline' width="600">

This notebook is part of the [Machine Learning School](https://www.ml.school) program.

Let's ensure we are running the latest version of the SakeMaker SDK. **Restart the Kernel** after you run the following cell.

In [2]:
!pip install -q --upgrade pip
!pip install -q --upgrade awscli boto3
!pip install -q --upgrade scikit-learn==0.23.2
!pip install -q --upgrade PyYAML==6.0
!pip install -q --upgrade sagemaker
!pip install -U sagemaker
!pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.171.0 requires PyYAML==6.0, but you have pyyaml 5.4.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.2 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Name: sagemaker
Version: 2.171.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, cloudpickle, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, PyYAML, schema, smdebug-rulesconfig, t

In [38]:
%load_ext autoreload
%autoreload 2

import sys
CODE_FOLDER = "code"
sys.path.append(f"./{CODE_FOLDER}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Session 1

In [39]:
from session1 import *

In [40]:
%%writefile {CODE_FOLDER}/session2.py

from session1 import *

from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep

Overwriting code/session2.py


## Step 1 - Training the Model

This script is responsible for training a simple neural network on the train data, validating the model, and saving it so we can later use it.

In [41]:
%%writefile {CODE_FOLDER}/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list: 
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting code/train.py


## Step 2 - Testing the Training Script

Let's test the script we just created by running it locally.

In [42]:
import os
import tempfile
from preprocessor import preprocess
from train import train

with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_directory=directory, 
        data_filepath=LOCAL_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )

CLASSES ['Adelie' 'Chinstrap' 'Gentoo']
Epoch 1/10
8/8 - 1s - loss: 0.8899 - accuracy: 0.4686 - val_loss: 0.8547 - val_accuracy: 0.4706
Epoch 2/10
8/8 - 0s - loss: 0.8511 - accuracy: 0.5063 - val_loss: 0.8217 - val_accuracy: 0.5294
Epoch 3/10
8/8 - 0s - loss: 0.8205 - accuracy: 0.5690 - val_loss: 0.7952 - val_accuracy: 0.6078
Epoch 4/10
8/8 - 0s - loss: 0.7958 - accuracy: 0.6360 - val_loss: 0.7724 - val_accuracy: 0.6667
Epoch 5/10
8/8 - 0s - loss: 0.7739 - accuracy: 0.6778 - val_loss: 0.7522 - val_accuracy: 0.7059
Epoch 6/10
8/8 - 0s - loss: 0.7547 - accuracy: 0.7238 - val_loss: 0.7335 - val_accuracy: 0.7059
Epoch 7/10
8/8 - 0s - loss: 0.7367 - accuracy: 0.7406 - val_loss: 0.7153 - val_accuracy: 0.7451
Epoch 8/10
8/8 - 0s - loss: 0.7195 - accuracy: 0.7615 - val_loss: 0.6977 - val_accuracy: 0.7843
Epoch 9/10
8/8 - 0s - loss: 0.7026 - accuracy: 0.7741 - val_loss: 0.6803 - val_accuracy: 0.8039
Epoch 10/10
8/8 - 0s - loss: 0.6855 - accuracy: 0.7950 - val_loss: 0.6627 - val_accuracy: 0.8235

INFO:tensorflow:Assets written to: /tmp/tmppmpayp8t/model/001/assets


## Step 3 - Setting up a Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. This Training Step will create a SageMaker Training Job in the background, run the training script, and upload the output to S3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information. 

SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the built-in [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the training script we wrote before. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available framework versions for each region. Here, you can also check the available SageMaker [Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md).

Notice the list of hyperparameters defined below. SageMaker will pass these hyperparameters as arguments to the entry point of the training script.

In [43]:
%%writefile -a {CODE_FOLDER}/session2.py

estimator = TensorFlow(
    entry_point=f"{CODE_FOLDER}/train.py",
    
    hyperparameters={
        "epochs": 50,
        "batch_size": 32
    },
    
    framework_version="2.6",
    instance_type="ml.m5.large",
    py_version="py38",
    instance_count=1,
    script_mode=True,
    
    # The default profiler rule includes a timestamp which will change each time
    # the pipeline is upserted, causing cache misses. Since we don't need
    # profiling, we can disable it to take advantage of caching.
    disable_profiler=True,

    role=role,
)

Appending to code/session2.py


We can now create the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) using the estimator we defined before.

This step will receive the train and validation split from the preprocessing step as inputs. Notice how we reference both splits using preprocess step. This creates a dependency between the Training and Processing Steps.

In [44]:
%%writefile -a {CODE_FOLDER}/session2.py

train_model_step = TrainingStep(
    name="train-model",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

Appending to code/session2.py


## Step 4 - Setting up a Tuning Step

Let's now create a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning). This Tuning Step will create a SageMaker Hyperparameter Tuning Job in the background and use the training script to train different model variants and choose the best one. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation for more information.

The Tuning Step requires a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) reference to configure the Hyperparameter Tuning Job.

Here is the configuration that we'll use to find the best model:

1. `objective_metric_name`: This is the name of the metric the tuner will use to determine the best model.
2. `objective_type`: This is the objective of the tuner. Should it "Minimize" the metric or "Maximize" it? In this example, since we are using the validation accuracy of the model, we want the objective to be "Maximize." If we were using the loss of the model, we would set the objective to "Minimize."
3. `metric_definitions`: Defines how the tuner will determine the metric's value by looking at the output logs of the training process.

The tuner expects the list of the hyperparameters you want to explore. You can use subclasses of the [Parameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange) class to specify different types of hyperparameters. This example explores different values for the `epochs` hyperparameter.

Finally, you can control the number of jobs and how many of them will run in parallel using the following two arguments:

* `max_jobs`: Defines the maximum total number of training jobs to start for the hyperparameter tuning job.
* `max_parallel_jobs`: Defines the maximum number of parallel training jobs to start.

In [45]:
%%writefile -a {CODE_FOLDER}/session2.py

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 50),
}

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

Appending to code/session2.py


We can now create the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep). 

This step will use the tuner we configured before and will receive the train and validation split from the preprocessing step as inputs.

In [46]:
%%writefile -a {CODE_FOLDER}/session2.py

tune_model_step = TuningStep(
    name = "tune-model",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_data_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

Appending to code/session2.py


## Step 5 - Switching Between Training and Tuning

We could use a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) or use a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) to create the model.

In this notebook, we will alternate between both methods and use the `USE_TUNING_STEP` flag to indicate which approach we want to run.

In [47]:
USE_TUNING_STEP = False

## Step 6 - Running the Pipeline

We can now define and run the SageMaker Pipeline using the Training or Tuning Step.

In [48]:
from session2 import *
from sagemaker.workflow.pipeline import Pipeline


pipeline = Pipeline(
    name="penguins-session2-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        train_dataset_baseline_destination,
        test_dataset_baseline_destination,
    ],
    steps=[
        preprocess_data_step, 
        tune_model_step if USE_TUNING_STEP else train_model_step
    ],
    pipeline_definition_config=pipeline_definition_config
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist or update it if it does.

In [49]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
